# Voltage 1

M&I 

Ch. 16 Electric Potential

- §16.1 &ndash; A Review of Potential Energy
- §16.3 &ndash; Potential Difference in a Uniform Field
- §16.5 &ndash; Potential Difference in a Non-Uniform Field
- §16.6 &ndash; Path Independence

In [1]:
########
## Env Setup / Preamble
########
%load_ext autoreload
%autoreload complete 

%load_ext Cython

%display latex

################

import cython
from sage.all import *

################

import tempfile
import mmap
from multiprocessing import shared_memory

################

import numpy as np
import scipy.constants

np.random.seed( int(scipy.constants.golden_ratio * 1.e6) )

from numpy import vectorize
from numpy import r_, c_, s_

################

####    boiler-plate class for named constants
from sage.symbolic.constants import Constant
class ConstantFromExpression(Constant):
    def __init__(self, name, v):
        conversions = dict(maxima=repr(maxima.coerce(v)))
        Constant.__init__(self, name, conversions=conversions)
        self._v = v
    def __float__(self): return float(self._v)
    def __complex__(self): return complex(self._v)
    def _mpfr_(self, R): return R(self._v)
    def _real_double_(self, R): return R(self._v)

def NamedExpression(name, expr):
    return ConstantFromExpression(name,expr).expression()
####

import sage.symbolic.units as units     # <: TODO

π   = NamedExpression('π', scipy.constants.pi)          # pi
ε_0 = NamedExpression('ε_0', scipy.constants.epsilon_0) # electric permittivity of vacuum
μ_0 = NamedExpression('μ_0', scipy.constants.mu_0)      # magnetic permeability of vacuum

𝛋 = 1. / (4 * π * ε_0)      # Coulomb's constant in [ N·m²/C² ]
q = NamedExpression('q', -scipy.constants.elementary_charge)

ℎ_planck = NamedExpression('ℎ', scipy.constants.Planck)
ℏ = NamedExpression('ℏ', scipy.constants.hbar)

α = NamedExpression('α', scipy.constants.fine_structure)

m_e = NamedExpression('m_e', scipy.constants.electron_mass)
m_p = NamedExpression('m_p', scipy.constants.proton_mass)

################

Parallelism().set(nproc=8)
print(f"Python: {sys.version}")
version()


Python: 3.12.5 (main, Feb 25 2025, 12:37:41) [Clang 16.0.0 (clang-1600.0.26.6)]


'SageMath version 10.6, Release Date: 2025-03-31'

In [2]:
##  sanity check
import this


The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


---

<hr width="82%" style="border-top:2px dashed; border-bottom:none"/>

<hr width="64%" style="border-top:1px dotted; border-bottom:none"/>

<div style="page-break-after: always;"/> 
<h2 id='1' style="display:none">[1]</h2>

>   ---
>   ## [1]
>
>   A proton that is initially traveling at a speed of $110$ [m/s] enters a region
>   where there is an electric field. Under the influence of the electric field,
>   the proton slows down and comes to a stop. 
>
>   __What is the change in kinetic energy of the proton?__
>
>   &nbsp;&nbsp;&nbsp;&nbsp; $K = \fbox{\hspace{8em}?}\ \left[\mathrm{J}\right]$
>
>   __<p align='right'>(0.1 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<div style="page-break-after: always;"/> 
<h2 id='2' style="display:none">[2]</h2>

>   ---
>   ## [2]
>
>   
>
>   Locations $A$ and $B$ are in a region of uniform electric field, as shown. 
>
>   Along a path from $A$ to $B$, the change in potential is $1600$ [V].
>   The distance from $A$ to $B$ is $0.22$ [m].
>
>   __What is the magnitude of the electric field in this region?__
>
>   &nbsp;&nbsp;&nbsp;&nbsp; $\lvert\vec{E}\rvert = \fbox{\hspace{8em}?}$
>
>   (select one:)
>
>   - [ ] &nbsp; [V/m]
>
>   __<p align='right'>(0.1 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

- [x] [V/m]

<div style="page-break-after: always;"/> 
<h2 id='3' style="display:none">[3]</h2>

>   ---
>   ## [3]*
>
>   If the electric field exceeds about $3 \times 10^{6}$ [N/C] in air, a spark 
>   occurs. 
>
>   Approximately, __what is the maximum possible potential difference between
>   the plates of a capacitor__ whose gap is $3$ [mm], without causing a spark
>   in the air between them?
>
>   &nbsp;&nbsp;&nbsp;&nbsp; $\fbox{\hspace{8em}?}\ \left[\mathrm{V}\right]$
>
>   __<p align='right'>(10 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<div style="page-break-after: always;"/> 
<h2 id='4' style="display:none">[4]</h2>

>   ---
>   ## [4]*
>
>   In an old style television picture tube (not in a modern flat-panel TV),
>   electrons are boiled out of a very hot metal filament placed near a negative
>   metal plate (see figure).  
>
>   
>
>   These electrons start out nearly at rest, and are accelerated toward a 
>   positive metal plate. They pass through a hole in the positive plate on 
>   their way toward the picture screen. 
>
>   If the high-voltage supply in the television set maintains a potential
>   difference of 14,500 volts between the two plates, __what speed do the 
>   electrons reach?__
>
>   (You can use the non-relativisitic approximation here.)
>
>   &nbsp;&nbsp;&nbsp;&nbsp; $v = \fbox{\hspace{8em}?}$ &ensp; [m/s]
>
>   __<p align='right'>(0.1 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<div style="page-break-after: always;"/> 
<h2 id='5' style="display:none">[5]</h2>

>   ---
>   ## [5]
>
>   
>
>   As shown in the diagram, three large, thin, and uniformly charged, plates 
>   are arranged so that there are two adjacent regions of uniform electric 
>   field. The origin is at the center of the central plate. 
>
>   Location $A$ is at $\left\langle\ -0.4, 0, 0\ \right\rangle$ [m], and
>   location $B$ is at $\left\langle\ 0.6, 0, 0\ \right\rangle$ [m].
>
>   In region 1 the electric field is $\vec{E}_1 = \left\langle 525, 0, 0 \right\rangle$ [V/m],
>   in region 2, the electric field is $\vec{E}_2 = \left\langle -375, 0, 0 \right\rangle$ [V/m].
>
>   __<p align='right'>(0.1 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<h3 id='5a' style="display:none">(A)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x2075;&hairsp;&#x2E05;&hairsp;(A)
>
>   Consider a path from $A$ to $B$.
>
>   __Along this path, what is the change in electric potential?__
>
>   &emsp;&emsp;&emsp;&emsp; $\Delta V = \fbox{\hspace{8em}?}$
>
>   (select one:)
>
>   - [ ] &nbsp; [V]
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

- [x] [V]

<h3 id='5b' style="display:none">(B)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x2075;&hairsp;&#x2E05;&hairsp;(B)
>
>   __What is the change in electric potential along a path from $B$ to $A$?__
>
>   &emsp;&emsp;&emsp;&emsp; $\Delta V = \fbox{\hspace{8em}?}$
>
>   (select one:)
>
>   - [ ] &nbsp; [V]
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

- [x] [V]

<h3 id='5c' style="display:none">(C)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x2075;&hairsp;&#x2E05;&hairsp;(C)
>
>   There is a tiny hole in the central plate, so a moving particle can pass
>   through the hole. 
>
>   __If a proton started at $A$ and moved to $B$ along the path shown, what would
>   be the change in its kinetic energy?__
>
>   &emsp;&emsp;&emsp;&emsp; $\Delta K = \fbox{\hspace{8em}?}$
>
>   (select one:)
>
>   - [ ] &nbsp; [J]
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

- [x] [J]

<h3 id='5d' style="display:none">(D)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x2075;&hairsp;&#x2E05;&hairsp;(D)
>
>   If the proton had kinetic energy $6 \times 10^{-18}$ [J] when it is at 
>   location $A$, __what would the kinetic energy of the proton be upon reaching
>   location $B$?__
>
>   &emsp;&emsp;&emsp;&emsp; $\Delta K = \fbox{\hspace{8em}?}$
>
>   (select one:)
>
>   - [ ] &nbsp; [J]
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

- [x] [J]

<h3 id='5e' style="display:none">(E)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x2075;&hairsp;&#x2E05;&hairsp;(E)
>
>   __Would it be possible for the proton to actually reach location $B$?__
>
>   Note that if you find the final kinetic energy to be negative, that means 
>   the proton will not actually get to location $B$ (a negative kinetic energy 
>   is impossible).
>
>   (select one:)
>
>   - [ ] &nbsp; [J]
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

- [x] [J]

<div style="page-break-after: always;"/> 
<h2 id='6' style="display:none">[6]</h2>

>   ---
>   ## [6]
>
>   A dipole is centered at the origin, with its axis along the $y$ axis; so that
>   at locations on the $y$-axis, the electric field due to the dipole is given
>   by:
>   
>   $$
>       \vec{E} = 
>           \left\langle\ 
>               0,\ 
>               \left( 
>                   \frac{1}{4\pi\epsilon_0} (2q) \cdot \frac{s}{y^3}
>               \right),\ 
>               0\ 
>               \right\rangle 
>       \left[
>           \mathrm{V} / \mathrm{m}
>       \right]
>   $$
>
>   The charges making up the dipole are $q_1 = +4$ [nC] and $q_2 = -4$ [nC], 
>   and the dipole separation is $s = 9$ [mm] (see figure below).
>
>   __What is the potential difference; along a path starting at location 
>   $P_1 = \left\langle\ 0,\ 0.03,\ 0\ \right\rangle$ [m], and ending at 
>   location $P_2 = \left\langle\ 0, 0.06,\ 0\ \right\rangle$ [m]?__
>
>   &nbsp;&nbsp;&nbsp;&nbsp; $\fbox{\hspace{8em}?}\ \left[\mathrm{V}\right]$
>
>   __<p align='right'>(0.1 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<div style="page-break-after: always;"/> 
<h2 id='7' style="display:none">[7]</h2>

>   ---
>   ## [7]*
>
>   __Geiger Tube__
>
>   A Geiger Tube, used to detect radioactive decays, contains a long thin metal
>   wire, surrounded by a concentric long narrow metal tube.
>   Insulating spokes hold the wire in the center of the tube, and prevent 
>   electrical contact between the wire and the tube. 
>
>   A variable power supply is connected to the device as shown. Normally, no 
>   current flows, because the tube and wire are not connected to each other. 
>   The power supply maintains opposite charges on the wire and the tube.
>
>   
>
>   __<p align='right'>(10 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<h3 id='7a' style="display:none">(A)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x2077;&hairsp;&#x2E05;&hairsp;(A)
>
>   The electric field in the space between the wire and the outer tube is due 
>   only to the wire &mdash; the contribution due to the charges on the outer 
>   tube sum to zero! 
>
>   When the electric field at any location in air reaches $3 \times 10^6$ [V/m],
>   the air ionizes and becomes a conductor. 
>   
>   For the tube shown, the length $L = 76$ [cm], the inner radius $r = 0.6$ [mm],
>   and the outer radius $R = 3.5$ [cm]. 
>
>   __How much charge would there need to be on the inner wire, in order to 
>     ionize the air near the wire__ (where the field is largest)__?__
>
>   &emsp;&emsp;&emsp;&emsp; $Q = \fbox{\hspace{8em}?}\ \left[\textrm{C}\right]$
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

In [5]:
L = 76e-2
R_i = 0.6e-3
R_o = 3.5e-2
E = 3e6

Q = E * (2 * π * ε_0 * L) * (R_i)
print(n(Q))
Q


7.61052638442067e-8


2736.00000000000*ε_0*π

<h3 id='7b' style="display:none">(B)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x2077;&hairsp;&#x2E05;&hairsp;(B)
>
>   When there is enough charge on the wire to ionize the air, you see a glow in
>   the air, very near the inner wire. 
>
>   When this happens, __what is the absolute value of the potential difference 
>   along a path from the inner wire to the outer tube?__
>
>   (Note that the electric field in this region is _not_ uniform.)
>
>   &emsp;&emsp;&emsp;&emsp; $\left\lvert \Delta V \right\rvert = \fbox{\hspace{8em}?}\ \left[\textrm{V}\right]$
>
>   The Geiger Tube was one of the first electronic particle detectors. 
>   The voltage is set just below the threshold for making the air glow near the
>   wire (Part B). A charged particle that passes near the center wire can trigger 
>   ionization of the air, leading to a large current that can be easily measured. 
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

In [8]:
# TODO :> also derive this in cylindrical coords, later

ΔV = -(Q / (2*π*ε_0*L)) * ln(R_o / R_i)
print(n(abs(ΔV)))

abs(ΔV)


7319.11263345973


-7319.11263345973

<div style="page-break-after: always;"/> 
<h2 id='8' style="display:none">[8]</h2>

>   --- 
>   ## [8]
>
>   A capacitor consists of two charged disks of radius $R = 3.6$ [m], separated
>   by a distance $s = 2$ [mm] (see figure).
>
>   
>
>   The magnitude of the charge on each disk is $47$ [µC].
>
>   Consider points $A$, $B$, $C$, and $D$ inside the capacitor, as shown in the
>   diagram. 
>
>   The distance $s_1 = 1.3$ [mm], and the distance $s_2 = 0.7$ [mm].
>
>   (Assume the $+x$-axis to the right, $+y$-axis is up the page, and the $+z$-axis
>    is out of the page.)
>
>   __<p align='right'>(0.1 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<h3 id='8a' style="display:none">(A)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x2078;&hairsp;&#x2E05;&hairsp;(A)
>
>   __What is the electric field inside the capacitor?__
>
>   &emsp;&emsp;&emsp;&emsp; $\vec{E} = \left\langle\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?}\ \right\rangle \left[\mathrm{N}/\mathrm{C}\right]$
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='8b' style="display:none">(B)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x2078;&hairsp;&#x2E05;&hairsp;(B)
>
>   First, calculate the potential difference $V_B - V_A$.
>
>   __What is $\Delta \vec{l}$ along this path?__
>
>    &emsp;&emsp;&emsp;&emsp; $\Delta\vec{l} = \left\langle\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?}\ \right\rangle \left[\mathrm{m}\right]$
>
>   __What is $V_B - V_A$?__
>
>   &emsp;&emsp;&emsp;&emsp; $V_B - V_A = \fbox{\hspace{8em}?}\ \left[\mathrm{V}\right]$
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='8c' style="display:none">(C)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x2078;&hairsp;&#x2E05;&hairsp;(C)
>
>   Next, calculate the potential difference $V_C - V_B$.
>
>   __What is $\Delta \vec{l}$ along this path?__
>
>    &emsp;&emsp;&emsp;&emsp; $\Delta\vec{l} = \left\langle\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?}\ \right\rangle \left[\mathrm{m}\right]$
>
>   __What is $V_B - V_A$?__
>
>   &emsp;&emsp;&emsp;&emsp; $V_C - V_B = \fbox{\hspace{8em}?}\ \left[\mathrm{V}\right]$
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='8d' style="display:none">(D)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x2078;&hairsp;&#x2E05;&hairsp;(D)
>
>   Finally, calculate the potential difference $V_A - V_C$.
>
>   __What is $\Delta \vec{l}$ along this path?__
>
>    &emsp;&emsp;&emsp;&emsp; $\Delta\vec{l} = \left\langle\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?},\ \fbox{\hspace{4em}?}\ \right\rangle \left[\mathrm{m}\right]$
>
>   __What is $V_A - V_C$?__
>
>   &emsp;&emsp;&emsp;&emsp; $V_A - V_C = \fbox{\hspace{8em}?}\ \left[\mathrm{V}\right]$
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

---